Training a LSTM model with ULMFIT approach on original text and then on reversed text. Then ensembaling the predictions of both.

Inpiration source [here](https://www.kaggle.com/gurharkhalsa/backwards-forwards-ulmfit-ensemble).

In [ ]:
#!pip install --no-index --find-links="../input/commonlit-pkgs-offline-download" spacy==3.1.1
!pip install spacy==3.1.1

In [ ]:
#!pip install --no-index --find-links="../input/commonlit-pkgs-offline-download" fastai==2.4.1
!yes Y|conda install -c fastai fastai=2.4.1

## Import libraries

In [ ]:
import fastai
fastai.__version__

In [ ]:
from typing import *
from pandas.core.frame import DataFrame
from fastai.text.all import *

In [ ]:
path = Path('../input')
op_path = Path('/kaggle/working')
train_path = path/'commonlitreadabilityprize/train.csv'
test_path = path/'commonlitreadabilityprize/test.csv'
aug_path = path/'common-lit-datset-with-synonym-replacement/aug_df.csv'
cbt_path = path/'commonlit-infusing-data-with-same-domain-data/vanilla_comlit_train_infused_with_cbt.csv'
cbt_aug_df_path = path/'commonlit-infusing-data-with-same-domain-data/aug_comlit_infused_with_cbt.csv'

In [ ]:
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
df_all = pd.concat([df_train, df_test])
df_aug = pd.read_csv(aug_path)
df_cbt = pd.read_csv(cbt_path)
df_cbt_aug = pd.read_csv(cbt_aug_df_path)
df_cbt_subset = df_cbt[:8000]

In [ ]:
df_aug.head(1)

In [ ]:
df_cbt_aug.tail(1)

## Fill nan targets with median value

In [ ]:
df_cbt_aug['target'] = df_cbt_aug['target'].fillna(df_cbt_aug['target'].median())

In [ ]:
df_cbt_aug.tail(1)

In [ ]:
df_cbt_aug_subset = df_cbt_aug[:10000]

## The backwards model
Trained on text in reverse order

## The dataloader

In [ ]:
dls_lm_back = TextDataLoaders.from_df(df_cbt_subset, text_col='excerpt',
                                      is_lm=True, valid_pct=0.1,
                                      bs=256, seq_len=72, backwards=True)

In [ ]:
dls_lm_back.show_batch(min_n=3)

## The backwards model

In [ ]:
learn_back = language_model_learner(dls_lm_back,
                               AWD_LSTM,
                               drop_mult=2.0,
                               metrics=[accuracy, Perplexity()], 
                               path=path,
                              wd=0.5).to_fp16()

## Training the model

In [ ]:
learn_back.path = op_path

In [ ]:
learn_back.fit_one_cycle(1, 1e-2)

In [ ]:
def show_me_lrs(learn):
    suggestions = namedtuple('Suggestions', ["min", "steep",
                                            "valley", "slide"])
    lr_min, lr_steep,lr_valley, lr_slide = learn.lr_find(suggest_funcs=(minimum, steep,valley, slide))
    suggested_lrs = suggestions(lr_min, lr_steep, lr_valley, lr_slide)
    
    print(f"Minimum/10:\t{lr_min:.2e}\
          \nSteepest point:\t{lr_steep:.2e}\
          \nLongest valley:\t{lr_valley:.2e}\
          \nSlide interval:\t{lr_slide:.2e}")
    
    return suggested_lrs

In [ ]:
suggested_lrs = show_me_lrs(learn_back)

In [ ]:
learn_back.unfreeze()
learn_back.fit_one_cycle(3, suggested_lrs.slide)

good enough accuracy, let;s save it

In [ ]:
learn_back.path = Path(".")
learn_back.save_encoder('./back_final_encoder')

## Backward Text regressor

In [ ]:
data = DataBlock(
       blocks=(TextBlock.from_df('excerpt', 
                                 vocab=dls_lm_back.vocab, seq_len=72,
                                 backwards=True), RegressionBlock),
       get_x=ColReader('text'),get_y=ColReader('target'),
       splitter=TrainTestSplitter(test_size=0.2))

In [ ]:
#data.summary(df_train)

In [ ]:
dls_reg_back = data.dataloaders(df_cbt_aug_subset, bs=128)

## The regression model

In [ ]:
learn_reg_back = text_classifier_learner(dls_reg_back, AWD_LSTM, drop_mult=2.0,
                                opt_func=QHAdam,
                                metrics=rmse,
                                   wd=1.0).to_fp16()

In [ ]:
learn_reg_back.path = Path(".")
learn_reg_back = learn_reg_back.load_encoder('./back_final_encoder')

## Training backwards regresion model

In [ ]:
learn_reg_back.fit_one_cycle(1, 2e-2)

In [ ]:
suggested_lrs = show_me_lrs(learn_reg_back)

In [ ]:
learn_reg_back.freeze_to(-2)
learn_reg_back.fit_one_cycle(1, slice(1e-3/(2.6**4),1e-3))

In [ ]:
suggested_lrs = show_me_lrs(learn_reg_back)

In [ ]:
learn_reg_back.freeze_to(-3)
learn_reg_back.fit_one_cycle(1, slice(1e-3/(2.6**4),1e-3))

In [ ]:
suggested_lrs = show_me_lrs(learn_reg_back)

In [ ]:
learn_reg_back.unfreeze()
learn_reg_back.fit_one_cycle(10,slice(1e-3/(2.6**4),1e-3))

In [ ]:
learn_reg_back.export("./back_final_model")

## Create the forward model 

## The dataloader

In [ ]:
dls_lm_forward = TextDataLoaders.from_df(df_cbt_subset, text_col='excerpt',
                                      is_lm=True, valid_pct=0.1,
                                      bs=128, seq_len=72, backwards=False)

In [ ]:
dls_lm_forward.show_batch(min_n=3)

## The forward Language model

In [ ]:
learn_forward = language_model_learner(dls_lm_forward,
                               AWD_LSTM,
                               drop_mult=2.0,
                               metrics=[accuracy, Perplexity()], 
                               path=path,
                              wd=0.5).to_fp16()

In [ ]:
learn_forward.path = op_path

In [ ]:
learn_forward.fit_one_cycle(1, 2e-2)

In [ ]:
suggested_lrs = show_me_lrs(learn_forward)

In [ ]:
learn_forward.unfreeze()
learn_forward.fit_one_cycle(3, suggested_lrs.slide)

In [ ]:
learn_forward.path = Path(".")
learn_forward.save_encoder('./forward_final_encoder')

## Forward Text Regressor

In [ ]:
data = DataBlock(
       blocks=(TextBlock.from_df('excerpt', 
                                 vocab=dls_lm_forward.vocab, seq_len=72,
                                 backwards=False), RegressionBlock),
       get_x=ColReader('text'),get_y=ColReader('target'),
       splitter=TrainTestSplitter(test_size=0.2))

In [ ]:
#data.summary(df_aug)

## The forward dataloader

In [ ]:
dls_reg_forward = data.dataloaders(df_cbt_aug_subset, bs=128)

In [ ]:
dls_reg_forward.show_batch()

## The regression model

In [ ]:
learn_reg_forward = text_classifier_learner(dls_reg_forward, AWD_LSTM, drop_mult=2.0,
                                opt_func=QHAdam,
                                metrics=rmse,
                                   wd=1.0).to_fp16()

In [ ]:
learn_reg_forward.path = Path(".")
learn_reg_forward = learn_reg_forward.load_encoder('./forward_final_encoder')

## Training forward regression model

In [ ]:
learn_reg_forward.fit_one_cycle(1, 2e-2)

In [ ]:
suggested_lrs = show_me_lrs(learn_reg_forward)

In [ ]:
learn_reg_forward.freeze_to(-2)
learn_reg_forward.fit_one_cycle(1, slice(1e-3/(2.6**4),1e-3))

In [ ]:
suggested_lrs = show_me_lrs(learn_reg_forward)

In [ ]:
learn_reg_forward.freeze_to(-3)
learn_reg_forward.fit_one_cycle(1, slice(1e-3/(2.6**4),1e-3))

In [ ]:
suggested_lrs = show_me_lrs(learn_reg_forward)

In [ ]:
learn_reg_forward.unfreeze()
learn_reg_forward.fit_one_cycle(7, slice(1e-3/(2.6**4),1e-3))

In [ ]:
learn_reg_forward.export("./forward_final_model")